<a href="https://colab.research.google.com/github/lucasduartesobreira/1-Comp-de-Inteligencia-Computacional-2022-2/blob/main/1%C2%AA_Competi%C3%A7%C3%A3o_EC_2022_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competicao-um-ic/train.csv
/kaggle/input/competicao-um-ic/test.csv


In [ ]:
df_train = pd.read_csv('/kaggle/input/competicao-um-ic/train.csv')
df_test = pd.read_csv('/kaggle/input/competicao-um-ic/test.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
!pip install sweetviz
import sweetviz as sv

d_train = df_train
d_train['DS_STATUS_ITEM'] = d_train['DS_STATUS_ITEM'].replace({'Negado':1, 'Autorizado':0})

In [ ]:
my_report = sv.analyze(d_train, target_feat='DS_STATUS_ITEM')
my_report.show_html()
my_report.show_notebook()

In [ ]:
def preproc(df):
    df['DT_NASCIMENTO'] = df['DT_NASCIMENTO'].fillna(df['DT_NASCIMENTO'].median())

    df['DS_TIPO_ACOMODACAO'] = df['DS_TIPO_ACOMODACAO'].fillna('NAO SE APLICA')

    df['DS_TIPO_INTERNACAO'] = df['DS_TIPO_INTERNACAO'].fillna('NAO SE APLICA')

    df['DS_TIPO_CONSULTA'] = df['DS_TIPO_CONSULTA'].fillna('NAO SE APLICA')

    df['DS_UNIDADE_TEMPO_DOENCA'] = df['DS_UNIDADE_TEMPO_DOENCA'].fillna('dias')
    
    df['QT_TEMPO_DOENCA'] = df['QT_TEMPO_DOENCA'].fillna(0)
    
    df['QT_DIA_SOLICITADO'] = df['QT_DIA_SOLICITADO'].fillna(0)

    df['DS_TIPO_ATENDIMENTO'] = df['DS_TIPO_ATENDIMENTO'].fillna('DESCONHECIDO')
    
    df['DS_INDICACAO_ACIDENTE'] = df['DS_INDICACAO_ACIDENTE'].fillna('DESCONHECIDO')
    
    df['CD_GUIA_REFERENCIA'] = df['CD_GUIA_REFERENCIA'].fillna(0)
    
    return df

In [ ]:
def textcolumns(df):
    df['TAMANHO_DS_ITEM'] = df['DS_ITEM'].map(lambda s: len(str(s)))
    df['TAMANHO_DS_CLASSE'] = df['DS_CLASSE'].map(lambda s: len(str(s)))
    df['TAMANHO_DS_INDICACAO_CLINICA'] = df['DS_INDICACAO_CLINICA'].fillna('').map(lambda s: len(str(s)))
    
    return df


In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

def standardScalerFunc(data):
  ss = StandardScaler()
  sstransformed = ss.fit_transform(data)

  return ss, pd.DataFrame(sstransformed)

def oneHotEncoderFunc(data):
  ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
  ohetransformed = ohe.fit_transform(data)
  
  return ohe, pd.DataFrame(ohetransformed)

In [ ]:
df_train = preproc(df_train)
df_train = textcolumns(df_train)

features_deleted = ['NR_PRODUTO', 'DS_TIPO_SAIDA', 'NR_SEQ_REQUISICAO', 'NR_SEQ_ITEM', 'QT_TEMPO_DOENCA', 'DS_UNIDADE_TEMPO_DOENCA']
numeric = ['QT_SOLICITADA', 'DT_REQUISICAO', 'DT_NASCIMENTO']
categorical = ['DS_INDICACAO_ACIDENTE', 'DS_TIPO_ITEM', 'DS_TIPO_INTERNACAO', 'DS_TIPO_GUIA', 'DS_TIPO_ACOMODACAO', 'DS_TIPO_ATENDIMENTO', 'DS_TIPO_CONSULTA', 'DS_TIPO_PREST_SOLICITANTE', 'DS_GRUPO', 'DS_REGIME_INTERNACAO', 'TAMANHO_DS_INDICACAO_CLINICA', 'TAMANHO_DS_ITEM', 'TAMANHO_DS_CLASSE', 'DS_CBO']

df_train_numeric = df_train[numeric]
df_train_categorical = df_train[categorical]

target = df_train['DS_STATUS_ITEM']


X = df_train.drop(features_deleted, axis=1)
print(len(X))
y = target

227122


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # 80% training 20% test

#train
ss, x_train_ss = standardScalerFunc(X_train[numeric])
ohe, x_train_ohe = oneHotEncoderFunc(X_train[categorical])

x_train = pd.concat([x_train_ss, x_train_ohe], axis=1)

#test
x_test_ss = ss.transform(X_test[numeric])
x_test_ohe = ohe.transform(X_test[categorical])

x_test = pd.concat([pd.DataFrame(x_test_ss), pd.DataFrame(x_test_ohe)], axis=1)

#classifier
classifier = RandomForestClassifier(n_estimators=30, random_state=0, min_samples_split=4)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[28168  2664]
 [ 5467  9126]]
              precision    recall  f1-score   support

  Autorizado       0.84      0.91      0.87     30832
      Negado       0.77      0.63      0.69     14593

    accuracy                           0.82     45425
   macro avg       0.81      0.77      0.78     45425
weighted avg       0.82      0.82      0.82     45425

0.8210016510731976


In [ ]:
df_test_pp = preproc(df_test)
df_test_pp = textcolumns(df_test)
test_ss = ss.transform(df_test_pp[numeric])
test_ohe = ohe.transform(df_test_pp[categorical])

test = pd.concat([pd.DataFrame(test_ss), pd.DataFrame(test_ohe)], axis=1)

In [ ]:
pred = classifier.predict(test)

In [ ]:
from itertools import zip_longest
submission = pd.DataFrame.from_records(zip_longest(df_test['Unnamed: 0'], pred), columns=['ID', 'DS_STATUS_ITEM'])

submission.set_index('ID').to_csv('submission.csv')

In [ ]:
submission

,ID,DS_STATUS_ITEM
0,0,Negado
1,5,Autorizado
2,8,Negado
3,13,Autorizado
4,18,Negado
...,...,...
186139,413210,Autorizado
186140,413218,Autorizado
186141,413221,Autorizado
186142,413248,Autorizado
